In [1]:
import numpy as np
from chesslab_.utils import default_parameters as params
from chesslab_.utils import load_pkl
from chesslab_.training_torch import training
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn

In [2]:
np.random.seed(0)
torch.manual_seed(0)

batch_size = params.batch_size
#path='../new_encode/'
path = 'D:/database/ccrl/'
#path='E:/Archivos/new_pgn/'
#path="./tmp/"


name=f'{path}ccrl_states_elo2.pkl'
x_data = load_pkl(name)

name=f'{path}ccrl_results_elo2.pkl'
y_data = load_pkl(name)

print(x_data.shape)

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size = params.test_size, random_state = 0, shuffle = True)

del x_data
del y_data

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
save_name='./tmp/test_elo.0'

(6291280, 64)


In [3]:
lr = 0.1
mo = 0.1
start=0
epochs=30
# Instantiate model
print(lr)
encoding=training.encoding_2
for i in encoding.items(): print(i)

inter_encoding = {params.inter_map[i]:code for i,code in encoding.items()}
print(inter_encoding)

0.1
('.', tensor([0., 0., 0., 0.]))
('p', tensor([1., 0., 0., 0.]))
('P', tensor([0., 0., 0., 1.]))
('b', tensor([0., 1., 0., 0.]))
('B', tensor([0., 0., 1., 0.]))
('n', tensor([1., 1., 0., 0.]))
('N', tensor([0., 0., 1., 1.]))
('r', tensor([1., 0., 1., 0.]))
('R', tensor([0., 1., 0., 1.]))
('q', tensor([1., 0., 0., 1.]))
('Q', tensor([0., 1., 1., 0.]))
('k', tensor([1., 1., 1., 0.]))
('K', tensor([0., 1., 1., 1.]))
{0: tensor([0., 0., 0., 0.]), 1: tensor([1., 0., 0., 0.]), 2: tensor([0., 0., 0., 1.]), 3: tensor([0., 1., 0., 0.]), 4: tensor([0., 0., 1., 0.]), 5: tensor([1., 1., 0., 0.]), 6: tensor([0., 0., 1., 1.]), 7: tensor([1., 0., 1., 0.]), 8: tensor([0., 1., 0., 1.]), 9: tensor([1., 0., 0., 1.]), 10: tensor([0., 1., 1., 0.]), 11: tensor([1., 1., 1., 0.]), 12: tensor([0., 1., 1., 1.])}


In [4]:
class Model_0(nn.Module):

    def __init__(self):
        super().__init__()
        self.func_1=nn.ReLU()
        self.func_2=nn.ReLU()
        self.func_3=nn.ReLU()
        self.func_4=nn.ReLU()
        
        #self.d1 = nn.Dropout(0.5)
        #self.d2 = nn.Dropout(0.5)
        #self.d3 = nn.Dropout(0.5)
        #self.d4 = nn.Dropout(0.5)
        
        #self.b1= nn.BatchNorm2d(32)
        #self.b2= nn.BatchNorm2d(64)
        #self.b3= nn.BatchNorm2d(128)
        #self.b4 = nn.BatchNorm1d(256)
        
        self.cnn_1 = nn.Conv2d(4, 32, kernel_size=7,padding=3)
        self.cnn_2 = nn.Conv2d(32, 64, kernel_size=5,padding=2)
        self.cnn_3 = nn.Conv2d(64, 128, kernel_size=3,padding=1)

        self.linear_1 = nn.Linear(8*8*128,256 )
        self.linear_2 = nn.Linear(256, 2)

    def forward(self, x ):
        out = self.cnn_1(x)
        #out = self.b1(out)
        out = self.func_1(out)
        #out = self.d1(out)
        out = self.cnn_2(out)
        #out = self.b2(out)
        out = self.func_2(out)
        #out = self.d2(out)
        out = self.cnn_3(out)
        #out = self.b3(out)
        out = self.func_3(out)
        #out = self.d3(out)
        out = out.reshape([x.size(0), -1])
        out = self.linear_1(out)
        #out = self.b4(out)
        out = self.func_4(out)
        #out = self.d4(out)
        out = self.linear_2(out)

        return out

In [5]:

model = Model_0().cuda()

loss_fn = training.loss_fn
optim = training.optim

training.set_encoding(encoding)

train_loader=training.DataLoader( x_data = x_train,y_data=y_train,batch_size=batch_size,shuffle=True ,encoding = encoding )
test_loader=training.DataLoader( x_data = x_test,y_data=y_test,batch_size=batch_size,shuffle=False , encoding = encoding )

print(model)

Model_0(
  (func_1): ReLU()
  (func_2): ReLU()
  (func_3): ReLU()
  (func_4): ReLU()
  (cnn_1): Conv2d(4, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (cnn_2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (cnn_3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (linear_1): Linear(in_features=8192, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=2, bias=True)
)


In [6]:
for x,y in train_loader:
    print(x.shape, y.shape)
    model(x.to(device))
    break

torch.Size([128, 4, 8, 8]) torch.Size([128, 2])


In [7]:
inter_encoding

{0: tensor([0., 0., 0., 0.]),
 1: tensor([1., 0., 0., 0.]),
 2: tensor([0., 0., 0., 1.]),
 3: tensor([0., 1., 0., 0.]),
 4: tensor([0., 0., 1., 0.]),
 5: tensor([1., 1., 0., 0.]),
 6: tensor([0., 0., 1., 1.]),
 7: tensor([1., 0., 1., 0.]),
 8: tensor([0., 1., 0., 1.]),
 9: tensor([1., 0., 0., 1.]),
 10: tensor([0., 1., 1., 0.]),
 11: tensor([1., 1., 1., 0.]),
 12: tensor([0., 1., 1., 1.])}

In [8]:
training.train(start=start,epochs=epochs,train_loader=train_loader,test_loader=test_loader,
         device=device,model=model,optim=optim,lr=lr,mo=mo,
         loss_fn=loss_fn,save_name=save_name,encoding=encoding)

epoch 1
2021-09-06 12:57:11
 train: 100.0/100 - loss:0.5463 - acc:0.7113           
 test: 100/100 - loss:0.5201 - acc:0.7317           
Epoch: 001/10 | Time: 339s = 5.7m | Train loss: 0.5463 | Train acc: 0.7113 | Test loss: 0.5201 | Test acc: 0.7317

--------------------------------------------------------------------------------
epoch 2
2021-09-06 13:02:50
 train: 100.0/100 - loss:0.4903 - acc:0.7511           
 test: 100/100 - loss:0.4711 - acc:0.7628           
Epoch: 002/10 | Time: 339s = 5.6m | Train loss: 0.4903 | Train acc: 0.7511 | Test loss: 0.4711 | Test acc: 0.7628

--------------------------------------------------------------------------------
epoch 3
2021-09-06 13:08:29
 train: 100.0/100 - loss:0.4433 - acc:0.7825           
 test: 100/100 - loss:0.4220 - acc:0.7957           
Epoch: 003/10 | Time: 335s = 5.6m | Train loss: 0.4433 | Train acc: 0.7825 | Test loss: 0.4220 | Test acc: 0.7957

--------------------------------------------------------------------------------
e

In [9]:
training.train(epochs=20,train_loader=train_loader,test_loader=test_loader,
         device=device,model= model, load_model='tmp/test_elo.0.10.pt',
         save_name=save_name)

epoch 11
2021-09-06 14:03:38
 train: 100.0/100 - loss:0.1765 - acc:0.9256           
 test: 100/100 - loss:0.3255 - acc:0.8720           
Epoch: 011/30 | Time: 335s = 5.6m | Train loss: 0.1765 | Train acc: 0.9256 | Test loss: 0.3255 | Test acc: 0.8720

--------------------------------------------------------------------------------
epoch 12
2021-09-06 14:09:13
 train: 100.0/100 - loss:0.1707 - acc:0.9285           
 test: 100/100 - loss:0.3141 - acc:0.8826           
Epoch: 012/30 | Time: 340s = 5.7m | Train loss: 0.1707 | Train acc: 0.9285 | Test loss: 0.3141 | Test acc: 0.8826

--------------------------------------------------------------------------------
epoch 13
2021-09-06 14:14:54
 train: 100.0/100 - loss:0.1666 - acc:0.9305           
 test: 100/100 - loss:0.3230 - acc:0.8824           
Epoch: 013/30 | Time: 335s = 5.6m | Train loss: 0.1666 | Train acc: 0.9305 | Test loss: 0.3230 | Test acc: 0.8824

-------------------------------------------------------------------------------